In [19]:
#  Copyright (c) 2023 Snowflake Computing Inc. All rights reserved.

# Import python packages & establish session
import pandas as pd
import json
from PyPDF2 import PdfFileReader
from snowflake.snowpark.files import SnowflakeFile
from io import BytesIO
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session, DataFrame


In [30]:

def is_speaker(line):
    # Check for the word "operator" or sentences with less than 3 words and no punctuation
    return "operator" == line.lower() or (len(line.split()) < 3 and (any(char.isalpha() for char in line)))

def process_transcript_file(file_path):
    # Initialize empty lists to store data
    speakers = []
    transcript_contents = []
    line_numbers = []

    # Read the transcript file line by line
    with open(file_path, 'r') as file:
        current_speaker = None
        current_transcript = ""
        current_line = 0

        for line in file:
            line = line.strip()

            # Skip empty lines
            if not line:
                continue

            # Check if the line contains a speaker's name or the word "operator"
            if is_speaker(line):
                # Update the current speaker and reset the transcript
                current_speaker = line
                current_transcript = ""
            else:
                # Append the line to the current transcript
                # current_transcript += line + " "
                current_transcript = line
                current_line += 1

                speakers.append(current_speaker)
                transcript_contents.append(current_transcript)
                line_numbers.append(current_line)

                current_transcript = ""

    # Create a DataFrame from the collected data
    df = pd.DataFrame({'SPEAKER': speakers, 'TRANSCRIPT_CONTENTS': transcript_contents, 'LINE_NUMBER': line_numbers})

    return df


In [31]:
# Example usage:
file_path = 'call-transcript-hartford-q4-2023.txt'
transcript_df = process_transcript_file(file_path)

# Add in the TICKER
ticker = "HIG"
transcript_df['TICKER'] = ticker

# Display the resulting DataFrame
transcript_df

,SPEAKER,TRANSCRIPT_CONTENTS,LINE_NUMBER,TICKER
0,Operator,"Good morning. My name is Rob, and I will be yo...",1,HIG
1,Operator,"Thank you. Susan Spivak, Senior Vice President...",2,HIG
2,Susan Spivak,Good morning and thank you for joining us toda...,3,HIG
3,Susan Spivak,Just a few comments before Chris begins. Today...,4,HIG
4,Susan Spivak,Our commentary today includes non-GAAP financi...,5,HIG
...,...,...,...,...
145,Beth Costello,"Yes. I mean, again, I think we've mentioned PF...",146,HIG
146,Alex Scott,Okay. Thank you.,147,HIG
147,Operator,We have reached the end of our question-and-an...,148,HIG
148,Susan Spivak,Thank you so much for all joining us today. An...,149,HIG


In [22]:
######################################
# Connect to Snowflake
######################################

run_in_streamlit_in_snowflake = False

# Current Environment Details
print("Connecting to Snowflake.....\n")

if (run_in_streamlit_in_snowflake) :
    session = get_active_session()
else :
    # --> EDIT THIS WITH YOUR ACCOUNT & CREDS
    connection_parameters = json.load(open('connection.json'))
    session = Session.builder.configs(connection_parameters).create()

# Now, we should have a connection, so test it and show basic connection info

session.sql_simplifier_enabled = True



print("Connected Successfully!..\n\n")


Connecting to Snowflake.....

Connected Successfully!..




In [32]:
session.sql("CREATE DATABASE IF NOT EXISTS TRANSCRIPTS").collect()
session.sql("CREATE SCHEMA IF NOT EXISTS TRANSCRIPTS.DATA").collect()

session.use_database("transcripts")
session.use_schema("data")

snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(snowflake_environment[0][1]))
print('Database                    : {}'.format(snowflake_environment[0][2]))
print('Schema                      : {}'.format(snowflake_environment[0][3]))
print('Snowflake version           : {}'.format(snowflake_environment[0][4]))

User                        : JLEMMON
Role                        : SYSADMIN
Database                    : TRANSCRIPTS
Schema                      : DATA
Snowflake version           : 8.4.1


# We want to add the transcript into a table

In [36]:
transcripts_snpk_df = session.write_pandas(transcript_df, "TRANSCRIPTS", auto_create_table=True, overwrite=True)

print(transcripts_snpk_df.to_pandas())

           SPEAKER                                TRANSCRIPT_CONTENTS  \
0         Operator  Good morning. My name is Rob, and I will be yo...   
1         Operator  Thank you. Susan Spivak, Senior Vice President...   
2     Susan Spivak  Good morning and thank you for joining us toda...   
3     Susan Spivak  Just a few comments before Chris begins. Today...   
4     Susan Spivak  Our commentary today includes non-GAAP financi...   
..             ...                                                ...   
145  Beth Costello  Yes. I mean, again, I think we've mentioned PF...   
146     Alex Scott                                   Okay. Thank you.   
147       Operator  We have reached the end of our question-and-an...   
148   Susan Spivak  Thank you so much for all joining us today. An...   
149       Operator  This concludes today's conference call. Thank ...   

     LINE_NUMBER TICKER  
0              1    HIG  
1              2    HIG  
2              3    HIG  
3              4   

RAG Made Easy w/ Snowflake Cortex
========

Creating an end-to-end Retrieval Augmented Generation process (or RAG) directly in Snowflake.
1) Extract full text from PDF files using Snowpark.
2) Chunk those documents using Langchain in Snowpark.
3) Use Cortex to create embeddings of those chunks.
4) Use Vector Similarity to show the most similar chunk when prompting an LLM.

# =============================================================
# =============================================================
# =============================================================
# =============================================================
# =============================================================
# =============================================================
# =============================================================
# =============================================================
# =============================================================
# =============================================================
# =============================================================


In [35]:
# --Create the chunked version of the table
# CREATE OR REPLACE TABLE CHUNK_TEXT AS
# SELECT
#         relative_path,
#         func.*
#     FROM raw_text AS raw,
#          TABLE(chunk_text(raw_text)) as func;

In [38]:
print(session.sql("show tables").collect())

convert_to_embeddings_sql = """CREATE OR REPLACE TABLE TRANSCRIPT_VECTOR_STORE AS \
SELECT 
ticker, speaker, transcript_contents, line_number, 
snowflake.ml.embed_text('e5-base-v2', transcript_contents) as transcript_embedding
FROM transcripts;
"""

print(convert_to_embeddings_sql)

session.sql(convert_to_embeddings_sql).collect()

[Row(created_on=datetime.datetime(2024, 2, 5, 11, 43, 26, 890000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), name='TRANSCRIPTS', database_name='TRANSCRIPTS', schema_name='DATA', kind='TABLE', comment='', cluster_by='', rows=150, bytes=26624, owner='SYSADMIN', retention_time='1', automatic_clustering='OFF', change_tracking='OFF', search_optimization='OFF', search_optimization_progress=None, search_optimization_bytes=None, is_external='N', enable_schema_evolution='N', owner_role_type='ROLE', is_event='N', budget=None, is_hybrid='N', is_iceberg='N'), Row(created_on=datetime.datetime(2024, 2, 5, 9, 14, 50, 532000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), name='TRANSCRIPT_VECTOR_STORE', database_name='TRANSCRIPTS', schema_name='DATA', kind='TABLE', comment='', cluster_by='', rows=150, bytes=487424, owner='SYSADMIN', retention_time='1', automatic_clustering='OFF', change_tracking='OFF', search_optimization='OFF', search_optimization_progress=

[Row(status='Table TRANSCRIPT_VECTOR_STORE successfully created.')]

In [39]:
vector_df = session.table('TRANSCRIPT_VECTOR_STORE')

print(vector_df.select(["ticker", "speaker"]).to_pandas())


    TICKER        SPEAKER
0      HIG       Operator
1      HIG       Operator
2      HIG   Susan Spivak
3      HIG   Susan Spivak
4      HIG   Susan Spivak
..     ...            ...
145    HIG  Beth Costello
146    HIG     Alex Scott
147    HIG       Operator
148    HIG   Susan Spivak
149    HIG       Operator

[150 rows x 2 columns]


In [ ]:
# --Vector distance allows use to find the most similar chunk to a question
# SELECT EPISODE_NAME, CHUNK from LLM_DEMO.PODCASTS.VECTORIZE 
#             ORDER BY VECTOR_L2_DISTANCE(
#             snowflake.ml.embed_text('e5-base-v2', 
#             'What makes time perceived to be slower?'
#             ), CHUNK_EMBEDDING
#             ) limit 1
#         ;

In [ ]:
--Pass the chunk we need along with the prompt to get a better structured answer from the LLM
SELECT snowflake.ml.complete(
    'llama2-7b-chat', 
    CONCAT( 
        'Answer the question based on the context. Be concise.','Context: ',
        (
            SELECT chunk FROM LLM_DEMO.PODCASTS.VECTORIZE 
            ORDER BY vector_l2_distance(
            snowflake.ml.embed_text('e5-base-v2', 
            'How should I optimise my caffeine intake?'
            ), chunk_embedding
            ) LIMIT 1
        ),
        'Question: ', 
        'How should I optimise my caffeine intake?',
        'Answer: '
    )
) as response;

In [ ]:
import streamlit as st # Import python packages
from snowflake.snowpark.context import get_active_session
session = get_active_session() # Get the current credentials

st.title("Ask Your Data Anything :snowflake:")
st.write("""Built using end-to-end RAG in Snowflake with Cortex functions.""")

model = st.selectbox('Select your model:',('llama2-70b-chat','llama2-13b-chat','llama2-7b-chat'))

prompt = st.text_input("Enter prompt", placeholder="What makes time perceived to be slower?", label_visibility="collapsed")

quest_q = f'''
select snowflake.ml.complete(
    '{model}', 
    concat( 
        'Answer the question based on the context. Be concise.','Context: ',
        (
            select chunk from LLM_DEMO.PODCASTS.VECTORIZE 
            order by vector_l2_distance(
            snowflake.ml.embed_text('e5-base-v2', 
            '{prompt}'
            ), chunk_embedding
            ) limit 1
        ),
        'Question: ', 
        '{prompt}',
        'Answer: '
    )
) as response;
'''

if prompt:
    df_query = session.sql(quest_q).to_pandas()
    st.write(df_query['RESPONSE'][0])

#Create a Streamlit app and run this code. Adjust where needed.
#------

import streamlit as st # Import python packages
from snowflake.snowpark.context import get_active_session
session = get_active_session() # Get the current credentials

st.title("Ask Your Data Anything :snowflake:")
st.write("""Built using end-to-end RAG in Snowflake with Cortex functions.""")

model = st.selectbox('Select your model:',('llama2-70b-chat','llama2-13b-chat','llama2-7b-chat'))

with st.form("prompt", clear_on_submit=False):
    text, btn = st.columns([6, 1])
    prompt = text.text_input("Enter prompt", placeholder="What makes time perceived to be slower?", label_visibility="collapsed")
    submit = btn.form_submit_button("Submit", type="primary", use_container_width=True)

quest_q = f'''
select snowflake.ml.complete(
    '{model}', 
    concat( 
        'Answer the question based on the context. Be concise.','Context: ',
        (
            select chunk from LLM_DEMO.PODCASTS.VECTORIZE 
            order by vector_l2_distance(
            snowflake.ml.embed_text('e5-base-v2', 
            '{prompt}'
            ), chunk_embedding
            ) limit 1
        ),
        'Question: ', 
        '{prompt}',
        'Answer: '
    )
) as response;
'''

if submit and prompt:
    df_query = session.sql(quest_q).to_pandas()
    st.write(df_query['RESPONSE'][0])